In [1]:
import os
import sys
import datetime

#图像读取库
from PIL import Image
#矩阵运算库
import numpy as np
import tensorflow as tf
from tensorflow import keras

from keras.preprocessing import image

print(tf.__version__)
print(keras.__version__, keras.__path__)

2.3.0
2.4.0 ['/home/wilson/.local/lib/python3.8/site-packages/tensorflow/keras']


In [2]:
# 获取训练的图片列表和分类标签信息
def load_images(data_dir):
    fpaths = []
    labels = []
    
    for label in os.listdir(data_dir):
        labelPath = os.path.join(data_dir, label)
        for fname in os.listdir(labelPath):
            # 文件名
            fpath = os.path.join(labelPath, fname)
            fpaths.append(fpath)
            
            # 标签
            labels.append(label_to_index[label])
    
    # 将图片名和标签 打包成一组，然后进行乱序
    temp = np.array([fpaths,labels])
    temp = temp.transpose()
    np.random.shuffle(temp)

    fpaths = list(temp[:,0])
    labels = list(temp[:,1])
    labels = [int(i) for i in labels]
    
    return fpaths, labels

def preprocess_image(image):
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.convert_image_dtype(image,dtype=tf.float32)
    image /= 255.0
    #print(333, image.shape, image)
    return image

def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    return preprocess_image(image)

# 从概率列表里，获取分类
def get_category_from_possiable(possiables):
    index = 0
    maxPoss = 0.0
    for k in range(len(possiables)):
        if possiables[k] > maxPoss:
            index = k
            maxPoss = possiables[k]
    return index

In [3]:
# 模型保存位置
model_save_path = 'emotion.model'

# 加载模型
model = None
if not os.path.exists(model_save_path):
    print("failed to find saved model in:", model_save_path)
    sys.exit(1)
else:
    model = keras.models.load_model(model_save_path)
    
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_192 (Functi (None, 6, 6, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1280)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 6405      
Total params: 2,264,389
Trainable params: 2,230,277
Non-trainable params: 34,112
_________________________________________________________________


In [4]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

# 分类
label_names = ["anger", "disgust", "fear", "happy", "sad"]
label_to_index = dict((name, index) for index, name in enumerate(label_names))

print(label_names)
print(label_to_index)

# 准备测试数据
data_dir = "/home/wilson/workstation/TFResFaceEmotion/emotions"

all_image_paths, all_image_labels = load_images(data_dir)
# all_image_paths = all_image_paths[:3]
# all_image_labels = all_image_labels[:3]

print("all_image_lavels", len(all_image_labels))
print("all_image_paths", len(all_image_paths))
path_ds = tf.data.Dataset.from_tensor_slices(all_image_paths)
image_ds = path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
#final_ds = image_ds.shuffle(buffer_size=10).batch(10)
final_ds = image_ds.batch(1)

predict = model.predict(final_ds)

correct = 0
for k in range(len(predict)):
    category = get_category_from_possiable(predict[k])
    if category == all_image_labels[k]:
        correct += 1
        print("predict: [{} {}/{}] {}".format(category, label_names[category], label_names[all_image_labels[k]], all_image_paths[k]))
    else:
        print("!!! predict: [{} {}/{}] {}".format(category, label_names[category], label_names[all_image_labels[k]], all_image_paths[k]))
    
print("correct {} {}/{}".format(correct/len(predict), correct, len(predict)))

['anger', 'disgust', 'fear', 'happy', 'sad']
{'anger': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'sad': 4}
all_image_lavels 79
all_image_paths 79
predict: [3 happy/happy] /home/wilson/workstation/TFResFaceEmotion/emotions/happy/happy (1).jpg
!!! predict: [3 happy/fear] /home/wilson/workstation/TFResFaceEmotion/emotions/fear/fear (11).jpg
predict: [3 happy/happy] /home/wilson/workstation/TFResFaceEmotion/emotions/happy/happy (26).jpg
!!! predict: [3 happy/sad] /home/wilson/workstation/TFResFaceEmotion/emotions/sad/sad (25).jpg
!!! predict: [3 happy/sad] /home/wilson/workstation/TFResFaceEmotion/emotions/sad/sad (7).jpg
predict: [2 fear/fear] /home/wilson/workstation/TFResFaceEmotion/emotions/fear/fear (6).jpg
predict: [3 happy/happy] /home/wilson/workstation/TFResFaceEmotion/emotions/happy/happy (30).jpg
predict: [3 happy/happy] /home/wilson/workstation/TFResFaceEmotion/emotions/happy/happy (22).jpg
!!! predict: [3 happy/sad] /home/wilson/workstation/TFResFaceEmotion/emotions/sad/sad (22)